<a href="https://colab.research.google.com/github/Private-Detective/CloudStream/blob/main/XL8R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <img src='https://www.libtorrent.org/img/logo-color-text.png' height='38'/>

In [ ]:
from IPython.display import clear_output
!pip install libtorrent
clear_output()
import os
import sys
import time
import libtorrent as lt
import requests

save_path = os.path.join(os.getcwd(), 'drive/MyDrive/Torrents')
torrent_file_name = os.path.join(save_path, 'downloaded_torrent.torrent')
link = ""  # @param {type:"string"}

if not os.path.exists(save_path):
    os.makedirs(save_path)

ses = lt.session()

try:
    if link.startswith("magnet:"):
        params = lt.parse_magnet_uri(link)
        params.save_path = save_path
        handle = ses.add_torrent(params)
    elif link.startswith("http://") or link.startswith("https://"):
        response = requests.get(link, stream=True)
        if response.status_code == 200:
            with open(torrent_file_name, 'wb') as f:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        f.write(chunk)
        else:
            raise ValueError("Failed to download file using requests. Status code: {}".format(response.status_code))

        torrent_info = lt.torrent_info(torrent_file_name)
        params = lt.add_torrent_params()
        params.ti = torrent_info
        params.save_path = save_path
        handle = ses.add_torrent(params)
    else:
        raise ValueError("Provided link is neither a magnet link nor a valid HTTP link.")

    start_time = time.time()

    while True:
        try:
            elapsed_time = time.time() - start_time
            status = handle.status()
            total_wanted_done = status.total_wanted_done

            download_speed = 0
            remaining_time_str = "Unknown"

            if total_wanted_done > 0:
                download_speed = (total_wanted_done / elapsed_time) / (1024 * 1024)
                remaining_time = (status.total_wanted - total_wanted_done) / download_speed
                remaining_time_str = time.strftime("%H:%M:%S", time.gmtime(remaining_time))

            print(f"\r{status.name[:18]}.. : {total_wanted_done / (1024 * 1024):.2f}MB "
                  f"[{elapsed_time:.0f} seconds, {download_speed:.2f}MB/s] "
                  f"Remaining time: {remaining_time_str} ({status.progress * 100:.2f}%)", end=' ')
            sys.stdout.flush()

            time.sleep(1)
        except KeyboardInterrupt:
            print("\nProcess stopped by user.")
            break

    ses.pause()
    partially_downloaded_file = os.path.join(save_path, handle.status().name)

except Exception as e:
    print(f"Error: {e}")

# <img src='https://storage.googleapis.com/pr-newsroom-wp/1/2018/11/Spotify_Logo_RGB_Green.png' height="39"/>

In [ ]:

from IPython.display import clear_output

# Check if spotdl is installed
SPOTDL = !(command -v "spotdl")
if not SPOTDL:
    !pip install spotdl
    clear_output()

# Add search query option
search_query = ""  # @param {type:"string"}
link = ""  # @param {type:"string"}
output = "/content/{title} - {artists}"  # kept the original output format

options = [
    "--format mp3",
    "--bitrate 128k",  # 128k as default bitrate
    "--lyrics musixmatch",  # Musixmatch as default lyrics provider
    "--sponsor-block",
    "--generate-lrc",
    "--print-errors",
    f"--output \"{output}\""
]

# Add link to spotdl command if provided
if link:
    !spotdl download {link} {" ".join(options)}
else:
    !spotdl download "{search_query}" {" ".join(options)}

try:
    # No need for eval or exec, just execute the command directly
    pass
except FileNotFoundError:
    print(f"Error: spotdl command not found. Please install spotdl.")
except Exception as e:
    print(f"Error: {e}")

# Add a success message
print("Download complete!")

# <img src='https://upload.wikimedia.org/wikipedia/commons/5/5f/FFmpeg_Logo_new.svg' height="34"/>

### *Encoder*

In [ ]:
#@title <h4> *Normal Encoder*

import subprocess

input_file = "" #@param {type:"string"}
output_file = "" #@param {type:"string"}
arguments = "" #@param {type:"string"}
ffpb = "/content/drive/MyDrive/ffpb.py"
!chmod 755 {ffpb}

while True:
    !"{ffpb}" -y -i "{input_file}" {arguments} "{output_file}"
    break

In [ ]:
#@title <h4> *H265 Encoder*
import subprocess
import sys
import os

input_dir = "/content/drive/MyDrive/DriveFire/Segments" #@param {type:"string"}
ffpb = "/content/drive/MyDrive/ffpb.py"
output_dir = "/content/drive/MyDrive/DriveFire/Vids/" #@param {type:"string"}

subprocess.check_output(['chmod', '755', ffpb])

for input_file in os.listdir(input_dir):
    input_file_path = os.path.join(input_dir, input_file)

    width = subprocess.check_output(['ffprobe', '-v', 'error', '-select_streams', 'v:0', '-show_entries', 'stream=width', '-of', 'csv=p=0', input_file_path]).decode('utf-8').strip()
    height = subprocess.check_output(['ffprobe', '-v', 'error', '-select_streams', 'v:0', '-show_entries', 'stream=height', '-of', 'csv=p=0', input_file_path]).decode('utf-8').strip()

    print(f"Processing video: {input_file}...")

    if int(width) == 1280 and int(height) < 600:
        size = "-s 1280:538"
    elif int(width) == 960 and int(height) < 500:
        size = "-s 960:458"
    elif int(width) == 854 and int(height) < 400:
        size = "-s 854:356"
    else:
        size = ""

    command = f"{ffpb} -n -i {input_file_path} -c:v libx265 {size} -c:a copy -c:s copy {output_dir}{input_file}"
    !{command}

### *Subtitle Encoder*

In [ ]:
input1 = "" #@param {type:"string"}
input2 = "" #@param {type:"string"}
filename = "" #@param {type:"string"}
!ffmpeg -loglevel error -i "$input1" -i "$input2" -c copy -map_metadata -1 -map_chapters -1 drive/MyDrive/"$filename".mkv

### *Video Splitter Tool*

In [ ]:
input_file = "/content/drive/MyDrive/Top.Gun.Maverick.2022.IMAX.720p.BluRay.900MB.x264-GalaxyRG.mkv" #@param {type:"string"}
segment_time = "00:15:00" #@param {type:"string"}
output_dir = "drive/MyDrive/DriveFire/Segments"

import os
import subprocess

if os.path.isfile(input_file):
    os.makedirs(output_dir, exist_ok=True)
    try:
        subprocess.run([
            "ffmpeg", "-y", "-i", input_file,
            "-segment_time", segment_time,
            "-f", "segment", "-c", "copy",
            "-map_chapters", "-1", "-map_metadata", "-1",
            f"{output_dir}/file%02d.mkv"
        ], check=True)
        print(f"Output directory contents:")
        !ls {output_dir}
    except subprocess.CalledProcessError as e:
        print(f"FFmpeg command failed with return code {e.returncode}.")
    except Exception as e:
        print(f"An error occurred: {e}")
else:
    print("Input file does not exist.")

In [ ]:
!printf "file '%s'\n" drive/MyDrive/DriveFire/Vids/*.mkv | sort -V > list.txt

In [ ]:
!ffmpeg -loglevel error -f concat -safe 0 -i list.txt -c copy -map 0 -map_metadata -1 -map_chapters -1 "drive/MyDrive/Nightcrawler.mkv"

In [ ]:
!cat list.txt

### *Multi-Image Generator*

In [ ]:
input_file = "" #@param {type:"string"}
seek_time = "" #@param {type:"string"}
num_of_frames = 10 #@param {type:"integer", default: 10}
increment = 1 #@param {type:"integer", default: 1}

import os
import glob
from IPython.display import Image, display, Markdown
import datetime

for filename in glob.glob('*.jpg'):
    os.remove(filename)

if ':' in seek_time:
    seek_time_dt = datetime.datetime.strptime(seek_time, "%H:%M:%S")
else:
    seek_time_dt = datetime.datetime(1900, 1, 1) + datetime.timedelta(seconds=int(seek_time))

for i in range(num_of_frames):
    timestamp_dt = seek_time_dt + datetime.timedelta(seconds=i*increment)
    timestamp = timestamp_dt.strftime("%H:%M:%S")
    command = f"ffmpeg -y -loglevel error -ss {timestamp} -i \"{input_file}\" -frames:v 1 /content/{timestamp}.jpg"
    os.system(command)

for filename in sorted(glob.glob('*.jpg'), key=lambda x: int(x[:-4].replace(':', ''))):
    display(Image(filename))
    display(Markdown(f"***{filename[:-4]}***"))

for filename in glob.glob('*.jpg'):
    os.remove(filename)

In [ ]:
import os

text = ""  #@param{type:"string"}
erase = False  #@param{type:"boolean"}
stack_file = "drive/MyDrive/stack.txt"

def load_stack():
    if os.path.exists(stack_file):
        with open(stack_file, 'r') as f:
            return [line.strip() for line in f]
    return []

def save_stack(stack):
    with open(stack_file, 'w') as f:
        f.writelines(f"{item}\n" for item in stack)

stack = load_stack()

if erase:
    stack.clear()
else:
    if text:
        stack.append(text)

save_stack(stack)

for item in stack:
    print(item)

### *Image Generator*

In [ ]:
import os
import re
import subprocess
from IPython.display import Image, display, Markdown

input_file = "" #@param{type:"string"}
seek_normal = "" #@param{type:"string"}
filter = "" #@param{type:"string"}
first_frame = False #@param{type:"boolean"}
last_frame = False #@param{type:"boolean"}

def get_video_duration(input_file):
    return int(float(subprocess.check_output(
        ["ffprobe", "-v", "error", "-show_entries", "format=duration", "-of", "default=noprint_wrappers=1:nokey=1", input_file]
    ).strip()))

def convert_to_seconds(time_str):
    if re.match(r'^\d{1,2}:\d{2}:\d{2}$', time_str):
        h, m, s = map(float, time_str.split(':'))
        return h * 3600 + m * 60 + s
    return float(time_str)

def extract_frame(input_file, output_file, ss=None, filter=None, label=None):
    command = []
    if ss:
        command.append(f"-ss {ss}")
    command.append(f"-i {input_file}")
    if filter:
        command.append(f"-vf \"{filter}\"")
    command.append("-vframes 1")
    command.append(output_file)

    command_str = " ".join(command)
    !ffmpeg -loglevel error -y {command_str}

    if os.path.exists(output_file) and label:
        with open(output_file, "rb") as f:
            display(Image(data=f.read(), format="jpg"))
            display(Markdown(label))

def extract_last_frame(input_file, duration):
    output_file = "last.jpg"
    extract_frame(input_file, output_file, duration, label=f"***{duration // 3600:02}:{(duration % 3600) // 60:02}:{duration % 60:02}*** &nbsp;-&nbsp; *Last Frame*")
    for ts in range(duration - 1, -1, -1):
        if os.path.exists(output_file):
            break
        extract_frame(input_file, output_file, ts, label=f"***{ts // 3600:02}:{(ts % 3600) // 60:02}:{ts % 60:02}*** &nbsp;-&nbsp; *Last Frame*")

if os.path.exists(input_file):
    if first_frame:
        extract_frame(input_file, "first.jpg", label="***00:00:00*** &nbsp;-&nbsp; *First Frame*")
    if last_frame:
        duration = get_video_duration(input_file)
        extract_last_frame(input_file, duration)
    if seek_normal:
        seek_time_in_seconds = convert_to_seconds(seek_normal)
        extract_frame(input_file, "normal.jpg", seek_time_in_seconds, filter=filter if filter else None, label=f"***{seek_normal}*** &nbsp;-&nbsp; *Normal Frame*")

!rm -rf *.jpg

### *Trimmer*

In [ ]:
input_file = "" #@param {type:"string"}
output_file = "1trim" #@param {type:"string"}
start_time = "00:50:04" #@param {type:"string"}
end_time = "01:03:14" #@param {type:"string"}
re_encode = False #@param {type:"boolean"}

from datetime import datetime, timedelta

def parse_time(time):
    if time.isdigit():
        return timedelta(seconds=int(time))
    else:
        return datetime.strptime(time, "%H:%M:%S") - datetime.strptime("00:00:00", "%H:%M:%S")

start_delta = parse_time(start_time)
end_delta = parse_time(end_time)

duration = end_delta - start_delta

if start_delta:
    start_cmd = "-ss " + str(start_delta)
else:
    start_cmd = ""

if end_delta:
    if not re_encode:
        end_cmd = "-t " + str(duration)
    else:
        end_cmd = "-to " + str(end_delta)
else:
    end_cmd = ""

if not re_encode:
    !ffmpeg -y -loglevel error -i "$input_file" {start_cmd} {end_cmd} -avoid_negative_ts 1 -reset_timestamps 1 -map 0 -c copy "$output_file".mkv
else:
    !ffmpeg -y -loglevel error -i "$input_file" {start_cmd} {end_cmd} -crf 0 -c:a copy "$output_file".mkv

### *MediaInfo*

In [ ]:
import subprocess
import json
import os

input_file = "/content/drive/MyDrive/Nightcrawler.mkv" #@param {type:"string"}

if not input_file:
    print("Error: No input file provided.")
else:
    if not os.path.exists(input_file):
        print("Error: Input file does not exist.")
    else:
        file_size_mb = os.path.getsize(input_file) / (1024 * 1024)
        print(f"File size: {file_size_mb / 1024 if file_size_mb > 1024 else file_size_mb:.2f} {'GB' if file_size_mb > 1024 else 'MB'}")

        try:
            ffprobe_output = subprocess.check_output(f"ffprobe -v error -show_streams -show_format -print_format json \"{input_file}\"", shell=True).decode("utf-8")
            media_info = json.loads(ffprobe_output)

            print("Media Information:")
            for stream in media_info['streams']:
                print(f"  {stream['codec_type'].capitalize()}:")
                print(f"    Codec: {stream['codec_name']}")
                if stream['codec_type'] == 'video':
                    print(f"    Resolution: {stream['width']}x{stream['height']}")
                elif stream['codec_type'] == 'audio':
                    print(f"    Channels: {stream['channel_layout']}")

            duration_output = subprocess.check_output(f"ffprobe -v error -show_entries format=duration -sexagesimal -of default=noprint_wrappers=1:nokey=1 -i \"{input_file}\"", shell=True).decode("utf-8").strip()
            print(f"Duration: {duration_output}")

        except subprocess.CalledProcessError:
            print("Error: Unable to get media information using ffprobe.")

In [ ]:
!ffmpeg -loglevel error -i "$input_file" -vf cropdetect -f null - 2>&1 | awk '/crop/{print $NF}'| tail -n1

# <img src='https://biplobsd.github.io/RLabClone/img/title_youtube-dl.png' height="38"/>

In [ ]:
YTDL = !(command -v "yt-dlp")
if not YTDL:
    !curl -s -L https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp -o /usr/local/bin/yt-dlp
    !chmod a+rx /usr/local/bin/yt-dlp

URL = "https://youtu.be/UveyUexjxFs?si=EprksKtUbyWu_X66"  #@param {type:"string"}
quality = ''  #@param ["bv[ext=mp4]+ba[ext=m4a]", "0", ""] {allow-input: true}
output = "%(title)s.%(ext)s"  #@param ["%(title)s.%(ext)s"] {allow-input: true}

if quality:
    !yt-dlp --no-warnings -q --progress -f {quality} -o "$output" "$URL"
else:
    !yt-dlp --no-warnings -q --progress -F -o "$output" "$URL"

# <font size=5> ✦ *Colab Still Alive*

In [ ]:
some_str = ' ' * 5120000000000

In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
import IPython
from IPython.display import clear_output
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Clicked on the connect button");
     btn.click()
     }

   btn = document.getElementById('connect')
   if (btn != null){
     console.log("Clicked on the reconnect button");
     btn.click()
     }
  }

setInterval(ClickConnect,60000)
'''))

clear_output()

# <font size=6> Scratchpad

In [ ]:
#@markdown <font size=4>***Downloader***
import ipywidgets as widgets
from IPython.display import display
from google.colab import files
import os

file = ""  # @param{type:"string"}

if file and os.path.isfile(file):
    files.download(file)
else:
    def list_files(path):
        return [item for item in os.listdir(path) if not item.startswith('.')]

    current_path = '/content/'
    history = []

    item_dropdown = widgets.Dropdown(
        options=list_files(current_path),
        description="Select file:",
        layout=widgets.Layout(width='260px')
    )

    action_button = widgets.Button(description="Select", layout=widgets.Layout(width='125px', height='25px'))
    back_button = widgets.Button(description="Back", layout=widgets.Layout(width='125px', height='25px'))

    def handle_selection(b):
        global current_path  # Declare current_path as global
        selected_item = item_dropdown.value
        full_path = os.path.join(current_path, selected_item)
        if os.path.isdir(full_path):
            history.append(current_path)
            current_path = full_path
            item_dropdown.options = list_files(current_path)
            item_dropdown.value = None
        elif os.path.isfile(full_path):
            files.download(full_path)

    def handle_back(b):
        global current_path  # Declare current_path as global
        if history:
            current_path = history.pop()
            item_dropdown.options = list_files(current_path)
            item_dropdown.value = None

    action_button.on_click(handle_selection)
    back_button.on_click(handle_back)

    display(item_dropdown, action_button, back_button)

In [ ]:
#@markdown <font size=4>***Demucs***
from IPython.display import clear_output
input = "" #@param{type:"string"}
DEMUCS=!(command -v "demucs")
if not DEMUCS:
  !python3 -m pip install -U demucs
  clear_output()
!demucs "$input"